# Gradient Descent and Backpropagation
#### Author: Jay Mody
---
## To Do:
- rework intro

## Introduction

Gradient descent and backpropagation is often presented as a blackbox process. Often you're presented with the depiction of descending down a mountain, or propagating the error from the end of the start to the beggining. But how does it all actually work? How would you implement it in code? To understand and implement backpropagation gradient descent, we are going to need to understand the math behind it.

This notebook is meant designed for an audience that is already familiar with neural networks, so concepts like feed forward, hidden layers, and activation functions are assumed. Here's a summary of what you should already be familiar with:

Required Knowledge:
- Basic Python Skills
- Numpy
- Calculus (derivatives, gradients, chain rule)
- Linear Algebra (matrices, matrix multiplication)
- Basic Understanding of a fully connected neural network (feed forward, activations, error, weights, bias)

Let's start by defining a problem that we can use a neural network to solve.

---

## The Neural Network
Let's look at a simple example of a neural newtork with a single hidden layer. The feed forward process would look something like this:

\begin{align}
\large
X \xrightarrow[W^1X + B^1]{\text{linear}} h_i \xrightarrow[f(h_i)]{\text{activation}} h_o \xrightarrow[W^2h_o + B^2]{\text{linear}} y_i \xrightarrow[f(y_i)]{\text{activation}} \hat{y}
\end{align}

where:
- $X$ is the inputs ($x_1, x_2$)


- $W^1$ is the weight matrix connecting the input layer to the hidden layer
- $B^1$ is the bias vector for the input layer to the hidden layer
- $W^2$ is the weight matrix connecting the hidden layer to the output layer
- $B^2$ is the bias vector for the hidden layer to the output layer
- $f(x)$ is an activation function


- $h_i$ is the hidden layer inputs
- $h_o$ is the hidden layer outputs
- $y_i$ is the output layer inputs
- $\hat{y}$ is the predictions (output layer outputs)